# Preprocesamiento de texto y extracción de características en PySpark (para el análisis de sentimiento)

En el presente notebook, procesaremos un conjunto de datos compuestos por tweets con el propósito de realizar un análisis de sentimiento con el módulo de regresión logística en Apache Spark. Se utilizará Spark ML y Spark NLP para las etapas de transformación de los datos e entrenamiento y evaluación del modelo de aprendizaje automático.

## 1. Importar las librerías de Spark ML

In [ ]:
import findspark
findspark.init('/usr/local/spark') #Especificar la ruta de Apache Spark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.ml.feature import (
    RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, StringIndexer
)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## 2. Importar los datos

Importaremos datos del Twitter en csv conteniendo tweets en la columna `select_text` y clases en la columna `sentiment` que representan el sentimiento de cada tweet ("negativo", "neutral" o "positivo").

Dicho conjunto de datos fue utilizado en la competición **Tweet Sentiment Extraction** de Kaggle.

In [ ]:
# Iniciar SparkSession
spark = SparkSession \
        .builder \
        .getOrCreate()

# Leer los datos
df = spark\
     .read\
     .option("header", "true")\
     .option("inferSchema", True)\
     .csv("train.csv")\ # Especificar la ruta del archivo train.csv
     .select("selected_text", "sentiment")

spark.sparkContext.setLogLevel("ERROR")

# Reducir el número de Shuffle partitions para 5
spark.conf.set("spark.sql.shuffle.partitions", "5")

## 3. Realizar una limpieza en los datos

Antes de preprocesarlos, es necesario que nuestros datos estén limpios. Es imperativo explorar los datos para encontrar errores e imperfecciones que pueden tener un impacto negativo durante la etapa de preprocesamiento.

Es muy importante asegurarnos de que **no hayan valores faltantes o nulos en el conjunto de datos**. La presencia de estos valores es la causa frecuente de problemas cuando trabajamos con Spark ML.

In [ ]:
# Quitar los valores faltantes o nulos
df_cleaned = df\
            .dropna()\
            .select("selected_text","sentiment")

# Contar la cantidad de filas del conjunto de datos
df_cleaned.count()

## 4. Preprocesamiento de los datos con RegexTokenizer y StopWordsRemover

Utilizaremos los siguientes **tranformers** para preprocesar nuestros datos:
1. RegexTokenizer: transforma el texto en un conjunto de tokens (palabras) aplicando una regular expression (regex); y
2. StopWordsRemover: remueve los tokens frecuentes de cada texto.

In [ ]:
# Extraer las palabras de cada texto mediante una expresión regular (regex)
regextokenizer = RegexTokenizer(inputCol="selected_text", outputCol="words", pattern="\\W")

# Remover las palabras comúnes del texto
englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover()\
        .setStopWords(englishStopWords)\
        .setInputCol("words")\
        .setOutputCol("preprocessed")

# Construir la pipeline de preprocesamiento
pipeline = Pipeline(stages=[regextokenizer, stops])

In [ ]:
# Aplicar la pipeline de preprocesamiento
pipelineFit = pipeline.fit(df_cleaned)
countvectorizer_transformed = pipelineFit.transform(df_cleaned)

# Remover filas con arrays vacios
filtered = countvectorizer_transformed.filter(f.size('preprocessed') > 0)

# Seleccionar la variable de entrada y la variable de salida
preprocessed = filtered.select("sentiment", "preprocessed")

In [ ]:
preprocessed.show(10, False)

## 5. Extracción de características con el CountVectorizer

En un modelo de **TF-IDF**, el CountVectorizer puede ser utilizado para calcular el TF o *Term Frequency*. El TF es un vector que representaría la ocurrencia de cada palabra dentro de cada documento. El IDF intentar asignar pesos a cada elemento del TF. Las palabras que aparecen con mayor frecuencia en los documentos reciben un peso menor en comparación con la palabras menos comúnes en los documentos.

El CountVectorizer es un transformer que hace un recuento de cada palabra en el documento y los expresa en un vector escaso.

También hemos transformado la variable de salida a una representación numérica de las categorías mediante el StringIndexer.

#### 5.1. Construir la pipeline de extracción

In [ ]:
# Transformar la variable de entrada en vectores de representación mediante el CountVectorizer
cv = CountVectorizer()\
    .setInputCol("preprocessed")\
    .setOutputCol("TFOut")\
    .setVocabSize(500)\
    .setMinTF(1)\
    .setMinDF(2)

# Aplicar el IDF
idf = IDF()\
    .setInputCol("TFOut")\
    .setOutputCol("features")\
    .setMinDocFreq(2)

# Representar la variable de salida en términos numéricos
label_stringIdx = StringIndexer(inputCol = "sentiment", outputCol = "label")

# Construir la pipeline
pipeline = Pipeline(stages=[cv, idf, label_stringIdx])

#### 5.2 Aplicar la pipeline de transformación

In [ ]:
pipelineFit = pipeline.fit(preprocessed)
countvectorizer_transformed = pipelineFit.transform(preprocessed).select("features", "label")
countvectorizer_transformed.show(10, False)

#### 5.3 Crear un modelo de regresión logística y evaluarlo

In [ ]:
# Hacer el fit y transform
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(countvectorizer_transformed)
predictions = lrModel.transform(countvectorizer_transformed)

# Hacer el predict
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="f1")
print("La precisión del modelo es del {:0.2f}%".format(evaluator.evaluate(predictions)*100))

## 6. Extracción de características con el HashingTF

#### 6.1. Construir la pipeline de extracción

In [ ]:
# Transformar la variable de entrada en vectores de representación mediante el HashingTF
tf = HashingTF()\
    .setInputCol("preprocessed")\
    .setOutputCol("TFOut")\
    .setNumFeatures(10000)

# Aplicar el IDF
idf = IDF()\
    .setInputCol("TFOut")\
    .setOutputCol("features")\
    .setMinDocFreq(2)

# Representar la variable de salida en términos numéricos
label_stringIdx = StringIndexer(inputCol = "sentiment", outputCol = "label")

# Construir la pipeline
pipeline = Pipeline(stages=[tf, idf, label_stringIdx])

#### 6.2 Aplicar la pipeline de extracción

In [ ]:
pipelineFit = pipeline.fit(preprocessed)
hashingtf_transformed = pipelineFit.transform(preprocessed).select("features", "label")
hashingtf_transformed.show(10, False)

#### 6.3 Crear un modelo de regresión logística y evaluarlo

In [ ]:
# Hacer el fit y transform
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(hashingtf_transformed)
predictions = lrModel.transform(hashingtf_transformed)

# Evaluar con MulticlassClassificationEvaluator
print("La precisión del modelo es del {:0.2f}%".format(evaluator.evaluate(predictions)*100))

## 7. Extracción de características con el Word2Vec

#### 7.1. Construir y aplicar la pipeline de extracción

In [ ]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="preprocessed",
outputCol="features")

# Representar la variable de salida en términos numéricos
label_stringIdx = StringIndexer(inputCol = "sentiment", outputCol = "label")

# Aplicar la pipeline de extracción
pipeline = Pipeline(stages=[word2Vec, label_stringIdx])
pipelineFit = pipeline.fit(preprocessed)
word2vec_transformed = pipelineFit.transform(preprocessed).select("features", "label")
word2vec_transformed.show(10, False)

#### 7.2 Crear un modelo de regresión logística y evaluarlo

In [ ]:
# Hacer el fit y transform
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(word2vec_transformed)
predictions = lrModel.transform(word2vec_transformed)

# Evaluar con MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="f1")
print("La precisión del modelo es del {:0.2f}%".format(evaluator.evaluate(predictions)*100))

## 8 Consideraciones finales

En este notebook, hemos realizado el preprocesamiento de texto, extracción de características y entrenado un modelo de regresión logística para clasificar el sentimiento de cada tweet, utilizando puramente los módulos de Spark Machine Learning. Con transformaciones sencillas, hemos lograr entrenar un modelo con una precisión del +80% sobre los datos de entrenamiento.

Podríamos mejorar nuestras pipelines de procesamiento con Spark NLP, incorporando a nuestra pipeline otros procesadores como Stemmer y Lemmatizer para normalizar el texto.